In [91]:
import pandas as pd
import numpy as np

TRAIN_DATA='./train.csv'
TEST_DATA='./test.csv'
print(Train)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [92]:
def load_train_data():
    df = pd.read_csv(TRAIN_DATA)
    return df

def load_test_data():
    df = pd.read_csv(TEST_DATA)
    return df 

In [93]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

In [94]:
#学習データの読み込み
df = load_train_data()
#学習デー タの特徴量
x_train = df.drop(['Survived', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)
#学習データのラベル
y_train = df['Survived'].values

In [95]:
import sklearn.preprocessing as sp

In [96]:
x_train.columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'], dtype='object')

In [97]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [98]:
#性別を数字に変換→０(women)と１(men)に変換
le = sp.LabelEncoder()
le.fit(x_train.Sex.unique())
x_train.Sex = le.fit_transform(x_train.Sex)

In [99]:
x_train.Sex

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [100]:
#性別をonehot表現に(計２回データをいじって機械学習しやすい形へ変更する)
one = sp.OneHotEncoder()
enced = one.fit_transform(x_train.Sex.values.reshape(1, -1).transpose())

temp = pd.DataFrame(index=df.Sex.index, columns='Sex-' + le.classes_, data=enced.toarray())
enced_data = pd.concat([x_train, temp], axis=1)
del enced_data['Sex']
enced_data

,PassengerId,Pclass,Age,SibSp,Parch,Sex-female,Sex-male
0,1,3,22.0,1,0,0.0,1.0
1,2,1,38.0,1,0,1.0,0.0
2,3,3,26.0,0,0,1.0,0.0
3,4,1,35.0,1,0,1.0,0.0
4,5,3,35.0,0,0,0.0,1.0
...,...,...,...,...,...,...,...
886,887,2,27.0,0,0,0.0,1.0
887,888,1,19.0,0,0,1.0,0.0
888,889,3,NaN,1,2,1.0,0.0
889,890,1,26.0,0,0,0.0,1.0


In [101]:
#欠損値補完→AgeにNaNがあるためNaNがある行を削除or適当な推測値を入れる（今回はmeanで平均値を入れる）
#欠損値補完をし無いと学習が失敗する
#df.isnull().any(axis=0)
#df.isnull().sum()
#meanで補完（平均値）
from sklearn.impute import SimpleImputer
im = SimpleImputer(missing_values=np.NaN, strategy='mean', fill_value=None)
enced_data = im.fit_transform(enced_data)

In [102]:
enced_data

array([[  1.        ,   3.        ,  22.        , ...,   0.        ,
          0.        ,   1.        ],
       [  2.        ,   1.        ,  38.        , ...,   0.        ,
          1.        ,   0.        ],
       [  3.        ,   3.        ,  26.        , ...,   0.        ,
          1.        ,   0.        ],
       ...,
       [889.        ,   3.        ,  29.69911765, ...,   2.        ,
          1.        ,   0.        ],
       [890.        ,   1.        ,  26.        , ...,   0.        ,
          0.        ,   1.        ],
       [891.        ,   3.        ,  32.        , ...,   0.        ,
          0.        ,   1.        ]])

In [105]:
enced_data[888]
#Nanに値が入っていることを確認

array([889.        ,   3.        ,  29.69911765,   1.        ,
         2.        ,   1.        ,   0.        ])

In [106]:
#ロジステック回帰を使用
clf = LogisticRegression(random_state=0)
#学習
clf.fit(enced_data, y_train)

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=0)

In [107]:
#テストデータの読み込み
df_test = load_test_data()
x_test = df_test.drop(['Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)
x_test.Sex = le.fit_transform(x_test.Sex)
enced_test = one.fit_transform(x_test.Sex.values.reshape(1, -1).transpose())
temp_test = pd.DataFrame(index=df_test.Sex.index, columns='Sex-' + le.classes_, data=enced_test.toarray())
enced_data_test = pd.concat([x_test, temp_test], axis=1)
del enced_data_test['Sex']
enced_data_test
im = SimpleImputer(missing_values=np.NaN, strategy='mean', fill_value=None)
enced_data_test = im.fit_transform(enced_data_test)

#テスト予測
pred_test = clf.predict(enced_data_test)



In [108]:
enced_data_test[10]

array([902.        ,   3.        ,  30.27259036,   0.        ,
         0.        ,   0.        ,   1.        ])

In [82]:
pred_test

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [109]:
#テストの答え合わせ
df_result = pd.read_csv('./gender_submission.csv')
act = df_result['Survived']
ac = np.sum(pred_test == act) / len(pred_test)
print(ac)

0.9880382775119617


In [84]:
#98%の確率で予想ができている

In [85]:
x_train.columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'], dtype='object')

In [ ]:
#ロジステック回帰とは
#割合データや２値分類データに用いられる。２値分類問題の場合は、適合率や再現率といった指標でモデル評価する
#基本的に0〜1の範囲の収まる値を返してくれる
#0〜1に収まる目的変数に対して線形回帰式と同じような構造をロジット変換を施すことでロジステックの式を求める

In [112]:
ac = np.sum(pred_test == act)
ac

413

In [114]:
len(pred_test)

418